In [1]:
%load_ext autoreload
%autoreload 2
from napari.qt.threading import thread_worker
import control
import napari
import gui
import time
import numpy as np
import toml

s = """
[valves]
ip = "192.168.1.2"
device = "wago_valves"

[mux]
device = "mux"

  [mux.mapping]
  0_0 = 46
  0_1 = 47
  1_0 = 44
  1_1 = 45
  2_0 = 42
  2_1 = 43
  3_0 = 40
  3_1 = 41
  waste = 38
  io = 39

[mc]
device = "minichip"

  [mc.mapping]
  0_0 = 36
  0_1 = 37
  1_0 = 34
  1_1 = 35
  2_0 = 32
  2_1 = 33
  buttons = 31
  sandwiches = 30

[camera]
device = "zyla_camera"

[objective]
device = "ti2_objective"
states = [ "4x", "?", "10x", "??", "???", "????" ]
zooms = [ 4, 0, 10, 0, 0, 0 ]

[light]
device = "sola_light"
port = "COM3"

[down_filter]
device = "ti2_filter1"
states = [ "egfp", "cy5", "mcherry", "dapi", "tritca", "open2" ]

[down_shutter]
device = "ti2_shutter1"

[up_filter]
device = "ti2_filter2"

[up_shutter]
device = "ti2_shutter2"

[stage]
device = "asi_stage"
port = "COM4"

[focus]
device = "ti2_focus"

[lightpath]
device = "ti2_lightpath"
"""

conf = toml.loads(s)
c = control.load(conf, path="C:/Users/karlk/Micro-Manager-2.0")
c._core.setTimeoutMs(100000)
c.exposure = 100
c.objective = "4x"
c.z = 5000
c.binning = 2
c._core.setProperty("camera", "TriggerMode", "Software (Recommended for Live Mode)")

In [2]:
c.mux = "closed"

In [15]:
for t in range(30):
    time.sleep(1)
    for i in range(30, 48):
        c.valves[i] = "off"
    time.sleep(1)
    for i in range(30, 48):
        c.valves[i] = "on"

In [33]:
bbsa = 0
na = 1
ab = 2
pbs = 3
urea_1m = 4
urea_0m = 5

waste = 0

In [4]:
gui.live(c)

(Viewer(camera=Camera(center=(0.0, 1023.5, 1023.5), zoom=0.28759765625, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 2048.0, 1.0), (0.0, 2048.0, 1.0)), current_step=(1023, 1023), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'live' at 0x188c70f6c80>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x00000188B73BE7A0>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={}),
 <napari._qt.qthreading.GeneratorWorker at 0x188c716b9a0>)

In [ ]:
c.mux = "closed"
c.mc.snw = "closed"
c.mc.btn = "closed"
c.mc.io = "closed"

print('>> 1/18. Starting Device Patterning for devices {}. \
    Starting with all valves closed. NOTE: flow of non-biotinylated BSA \
    should have already been done')
print('2/18. Opening sandwiches, outlet, bBSA inlet, and waste. \
    Flushing bBSA through inlet tree to waste for 30s')
c.mc.io = waste
c.mc.snw = "open"

c.mux = bbsa
time.sleep(5 * 60)

print('3/18. Done Flushing bBSA to waste. Flushing bBSA through \
    devices with buttons closed for 5min')
c.mc.io = "open"
time.sleep(5 * 60)

print('4/18. Opened buttons with bBSA flowing through devices to waste for 35min')
c.mc.btn = "open"
time.sleep(35 * 60)

print('5/18. Done Flowing bBSA through devices and closed inlet. \
    Flushing PBS through inlet tree to waste for 30s')
c.mc.io = waste
c.mux = pbs
time.sleep(5 * 60)

print('6/18. Done flowing PBS to waste. Flushing PBS through device \
    with  buttons open for 10min')
c.mc.io = "open"
time.sleep(10 * 60)

print('7/18. Done flushing PBS through devices. \
    Flowing neutravidin through inlet tree to waste for 30s')
c.mc.io = waste
c.mux = na
time.sleep(5 * 60)

print('8/18. Done flushing Neutravidin to waste. \
    Flowing Neutravidin through devices with buttons open for 30min')
c.mc.io = "open"
time.sleep(30 * 60)

print('9/18. Done flowing Neutravidin through devices. \
    Flowing PBS through devices with buttons open for 10min')
c.mux = pbs
time.sleep(15 * 60)

print('10/18. Done flowing PBS through devices and closed buttons. \
    Flowing bBSA through the device for another 35min (quench walls only)')
c.mc.btn = "closed"
c.mux = bbsa
time.sleep(35 * 60)

print('11/18. Done flowing bBSA through devices. \
    Flowing PBS through the device for 10min. **NEXT STEP IS ANTIBODY FLOWING**')
c.mux = pbs
time.sleep(15 * 60)

print('12/18. Done flowing PBS through devices and closed inlet. \
    Flowing Antibody through inlet tree to waste for 30s')
c.mc.io = waste
c.mux = ab
time.sleep(5 * 60)
        
print('13/18. Done flowing Antibody through inlet tree. Flowing \
    Antibody through device for 2min')
c.mc.io = "open"
time.sleep(2 * 60)

print('14/18. While flowing Antibody through devices, opened buttons. \
    Flowing for 13.3min')
c.mc.btn = "open"
time.sleep(800)

print('15/18. Closed buttons while flowing Antibody through device for 30s')
c.mc.btn = "closed"
time.sleep(30)

print('16/18. Done flowing Antibody through device. Flowing PBS through \
    inlet tree to waste for 30s')
c.mc.io = waste
c.mux = pbs
time.sleep(5 * 60)      

print('17/18. Done flowing PBS to waste. Flowing PBS through device for 10min')
c.mc.io = "open"
time.sleep(10 * 60)

print('18/18. Closed the outlets')
c.mc.io = "closed"
c.mux = "closed"
print('>> Done with device patterning')


>> 1/18. Starting Device Patterning for devices {}.     Starting with all valves closed. NOTE: flow of non-biotinylated BSA     should have already been done
2/18. Opening sandwiches, outlet, bBSA inlet, and waste.     Flushing bBSA through inlet tree to waste for 30s
3/18. Done Flushing bBSA to waste. Flushing bBSA through     devices with buttons closed for 5min
4/18. Opened buttons with bBSA flowing through devices to waste for 35min
5/18. Done Flowing bBSA through devices and closed inlet.     Flushing PBS through inlet tree to waste for 30s
6/18. Done flowing PBS to waste. Flushing PBS through device     with  buttons open for 10min
7/18. Done flushing PBS through devices.     Flowing neutravidin through inlet tree to waste for 30s
8/18. Done flushing Neutravidin to waste.     Flowing Neutravidin through devices with buttons open for 30min
9/18. Done flowing Neutravidin through devices.     Flowing PBS through devices with buttons open for 10min
10/18. Done flowing PBS through dev

In [9]:
c.mc.btn

'open'

In [12]:
c.mc.btn = "closed"

# STANDARD CURVE

In [3]:
conc_200um = 1
conc_100um = 2
conc_50um = 4
conc_20um = 5
conc_10um = 6
conc_5um = 7
conc_2um = 8
conc_0um = 10

s = f"""
overlap = 0
[default]
light = 100
exposure = 100
down_shutter = "closed"

[channels]
[channels.dapi]
down_filter = "dapi"
down_shutter = "open"

[times]
[times.a]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_0um}
wait = "5min"

[times.b]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.c]
mc.btn = "open"
wait = "10s"

[times.0]
acq = true
wait = "0s"


[times.d]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_2um}
wait = "5min"

[times.e]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.f]
mc.btn = "open"
wait = "10s"

[times.2]
acq = true
wait = "0s"

[times.g]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_5um}
wait = "5min"

[times.h]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.i]
mc.btn = "open"
wait = "10s"

[times.5]
acq = true
wait = "0s"

[times.j]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_10um}
wait = "5min"

[times.k]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.l]
mc.btn = "open"
wait = "10s"

[times.10]
acq = true
wait = "0s"

[times.m]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_20um}
wait = "5min"

[times.n]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.o]
mc.btn = "open"
wait = "10s"

[times.20]
acq = true
wait = "0s"

[times.p]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_50um}
wait = "5min"

[times.q]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.r]
mc.btn = "open"
wait = "10s"

[times.50]
acq = true
wait = "0s"

[times.s]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_100um}
wait = "5min"

[times.t]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.u]
mc.btn = "open"
wait = "10s"

[times.100]
acq = true
wait = "0s"

[times.v]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_200um}
wait = "5min"

[times.w]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.x]
mc.btn = "open"
wait = "10s"

[times.200]
acq = true
wait = "0s"

[times.y]
mc.btn = "closed"
mc.snw = "closed"
mc.io = "closed"
mux = "closed"
wait = "0s"
"""
conf = toml.loads(s)
gui.acquire(c, file="C:/Users/karlk/Desktop/standards_albert", **conf)


# UREA ASSAY

In [4]:
c.mc.snw

'closed'

Setting light to 100 on <control.Control object at 0x000001DB441C2E90>
Setting exposure to 100 on <control.Control object at 0x000001DB441C2E90>
Setting down_shutter to closed on <control.Control object at 0x000001DB441C2E90>
Setting snw to closed on <control.Control object at 0x000001DB441C2E90>
{'light': 100, 'exposure': 100, 'down_shutter': 'closed', 'mc': {'snw': 'closed'}}
Setting light to 100 on <control.Control object at 0x000001DB441C2E90>
Setting exposure to 100 on <control.Control object at 0x000001DB441C2E90>
Setting down_shutter to closed on <control.Control object at 0x000001DB441C2E90>
{'light': 100, 'exposure': 100, 'down_shutter': 'closed'}


In [4]:
c.mc.snw

'open'

Setting light to 100 on <control.Control object at 0x0000019733FEE1A0>
Setting exposure to 100 on <control.Control object at 0x0000019733FEE1A0>
Setting down_shutter to closed on <control.Control object at 0x0000019733FEE1A0>
Setting snw to closed on <control.MiniChip object at 0x0000019733FEDD50>
{'light': 100, 'exposure': 100, 'down_shutter': 'closed', 'mc': {'snw': 'closed'}}
Setting light to 100 on <control.Control object at 0x0000019733FEE1A0>
Setting exposure to 100 on <control.Control object at 0x0000019733FEE1A0>
Setting down_shutter to closed on <control.Control object at 0x0000019733FEE1A0>
{'light': 100, 'exposure': 100, 'down_shutter': 'closed'}
Setting light to 100 on <control.Control object at 0x0000019733FEE1A0>
Setting exposure to 100 on <control.Control object at 0x0000019733FEE1A0>
Setting down_shutter to open on <control.Control object at 0x0000019733FEE1A0>


In [12]:
c.mc.snw

'open'

In [11]:
### import uuid

c.mc.io = "open"
s = f"""
overlap = 0
[default]
light = 100
exposure = 100
down_shutter = "closed"

[channels]
[channels.dapi]
down_filter = "dapi"
down_shutter = "open"
"""
conf = toml.loads(s)
times = {}
ureas = [3, 9, 11]
buffer = 10
for i, urea in enumerate([0, 1, 0, 1, 0, 2, 0]):
    if i >= 4:
        iters = 2
    else:
        iters = 3
    for j in range(iters):
        times[f"{i}-{j}-a"] = {"mux": ureas[urea], "mc": {"snw": "open"}, "wait": "5min"}
        times[f"{i}-{j}-b"] = {"mc": {"snw": "closed"}, "wait": "10sec"}
        times[f"{i}-{j}-c"] = {"mc": {"btn": "open"}, "wait": "0s"}
        for k in range(60): 
            times[f"{i}-{j}-{k}"] = {"acq": True, "wait": "1min"}
        if i in [1, 5]:
            times[f"{i}-{j}-d"] = {"mux": buffer, "wait": "5min", "mc": {"snw": "open"}}
            times[f"{i}-{j}-e"] = {"wait": "1hr", "mc": {"snw": "closed"}}
        times[f"{i}-{j}-f"] = {"mc": {"btn": "closed"}, "wait": "0s"}
conf["times"] = times
gui.acquire(c, file="C:/Users/karlk/Desktop/ureas_albert", **conf)

In [ ]:
urea_0M = 9
urea_1M = 11
urea_2M = 12

s = f"""
overlap = 0
[default]
light = 100
exposure = 100
down_shutter = "closed"

[channels]
[channels.dapi]
down_filter = "dapi"
down_shutter = "open"

[times]
[times.a]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {urea_0M}
wait = "5min"

[times.b]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.c]
mc.btn = "open"
wait = "10s"

[times.0]
acq = true
wait = "0s"


[times.d]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_2um}
wait = "5min"

[times.e]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.f]
mc.btn = "open"
wait = "10s"

[times.2]
acq = true
wait = "0s"

[times.g]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_5um}
wait = "5min"

[times.h]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.i]
mc.btn = "open"
wait = "10s"

[times.5]
acq = true
wait = "0s"

[times.j]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_10um}
wait = "5min"

[times.k]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.l]
mc.btn = "open"
wait = "10s"

[times.10]
acq = true
wait = "0s"

[times.m]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_20um}
wait = "5min"

[times.n]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.o]
mc.btn = "open"
wait = "10s"

[times.20]
acq = true
wait = "0s"

[times.p]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_50um}
wait = "5min"

[times.q]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.r]
mc.btn = "open"
wait = "10s"

[times.50]
acq = true
wait = "0s"

[times.s]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_100um}
wait = "5min"

[times.t]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.u]
mc.btn = "open"
wait = "10s"

[times.100]
acq = true
wait = "0s"

[times.v]
mc.btn = "closed"
mc.io = "open"
mc.snw = "open"
mux = {conc_200um}
wait = "5min"

[times.w]
mc.mux = "closed"
mc.snw = "closed"
mc.io = "closed"
wait = "10s"

[times.x]
mc.btn = "open"
wait = "10s"

[times.200]
acq = true
wait = "0s"

[times.y]
mc.btn = "closed"
mc.snw = "closed"
mc.io = "closed"
mux = "closed"
wait = "0s"
"""
conf = toml.loads(s)
gui.acquire(c, file="C:/Users/karlk/Desktop/ureas_albert", **conf)


In [3]:
c.mux = "closed"
c.mc.io = "closed"
c.mc.btn = "closed"

In [7]:
c.mc.btn

'closed'

In [46]:
c.mc.io = "closed"
c.mux = "closed"

SyntaxError: invalid decimal literal (2147623745.py, line 1)

In [37]:
gui.live(c)

(Viewer(camera=Camera(center=(0.0, 511.5, 511.5), zoom=0.5751953125, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1024.0, 1.0), (0.0, 1024.0, 1.0)), current_step=(511, 511), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'live' at 0x1b0a5132a40>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001B08A1EA7A0>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={}),
 <napari._qt.qthreading.GeneratorWorker at 0x1b107593eb0>)

In [38]:
c.down_filter = "open2"

In [71]:
c.mc.btn

'closed'

In [4]:
import xarray as xr


<xarray.Dataset>
Dimensions:  (channel: 1, time: 1, tile_row: 1, tile_col: 1, tile_y: 1024,
              tile_x: 1024)
Coordinates:
  * channel  (channel) <U4 'dapi'
  * time     (time) int32 15
Dimensions without coordinates: tile_row, tile_col, tile_y, tile_x
Data variables:
    tile     (channel, time, tile_row, tile_col, tile_y, tile_x) uint16 dask.array<chunksize=(1, 1, 1, 1, 1024, 1024), meta=np.ndarray>
Attributes:
    channel_states:  {'dapi': {'down_filter': 'dapi', 'down_shutter': 'open'}}
    default_state:   {'down_shutter': 'closed', 'exposure': 500, 'light': 100}
    overlap:         0
    time_states:     {'0': {'mc': {'btn': 'closed', 'io': 'closed', 'snw': 'c...

In [72]:
c.down_shutter = "closed"

In [6]:
napari.imshow(arr["tile"])

(Viewer(camera=Camera(center=(0.0, 511.5, 511.5), zoom=0.5751953125, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(0.0, 719.0, 0.0, 0.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=6, ndisplay=2, last_used=0, range=((0.0, 1.0, 1.0), (0.0, 1440.0, 1.0), (0.0, 1.0, 1.0), (0.0, 1.0, 1.0), (0.0, 1024.0, 1.0), (0.0, 1024.0, 1.0)), current_step=(0, 719, 0, 0, 511, 511), order=(0, 1, 2, 3, 4, 5), axis_labels=('0', '1', '2', '3', '4', '5')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x1d012f27a00>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001D005661F30>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_

In [4]:
import xarray as xr
arr = xr.open_zarr("C:/Users/karlk/Desktop/ureas_albert")

In [5]:
arr

<xarray.Dataset>
Dimensions:  (channel: 1, time: 1440, tile_row: 1, tile_col: 1, tile_y: 1024,
              tile_x: 1024)
Coordinates:
  * channel  (channel) <U4 'dapi'
  * time     (time) <U6 '0-0-0' '0-0-1' '0-0-2' ... '7-2-57' '7-2-58' '7-2-59'
Dimensions without coordinates: tile_row, tile_col, tile_y, tile_x
Data variables:
    tile     (channel, time, tile_row, tile_col, tile_y, tile_x) uint16 dask.array<chunksize=(1, 1, 1, 1, 1024, 1024), meta=np.ndarray>
Attributes:
    channel_states:  {'dapi': {'down_filter': 'dapi', 'down_shutter': 'open'}}
    default_state:   {'down_shutter': 'closed', 'exposure': 100, 'light': 100}
    overlap:         0
    time_states:     {'0-0-0': {'acq': True, 'wait': 60}, '0-0-1': {'acq': Tr...

In [3]:
gui.live(c)

(Viewer(camera=Camera(center=(0.0, 511.5, 511.5), zoom=0.5751953125, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1024.0, 1.0), (0.0, 1024.0, 1.0)), current_step=(511, 511), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'live' at 0x17291c36800>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x00000172844E6950>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={}),
 <napari._qt.qthreading.GeneratorWorker at 0x172923a7eb0>)